
# Used Car Market Analysis using Web Scraping (Cars24)

**Tools Used:** Python, Requests, Selenium, BeautifulSoup, Regex, Pandas  
**Website:** Cars24  
**Objective:** A comprehensive analysis of the used car market, focusing on area-specific trends versus pan-India data.


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import re

In [2]:
url = "https://www.cars24.com/buy-used-car/?f=businessVertical%3A%3D%3Ags"

-> above link is having all available Cars and with there RTO i.e places where cars avaliable for sale.

## Problem Statement

The objective of this project is to collect and analyze used car listings from Cars24 to understand pricing patterns, vehicle characteristics, and market trends in the Indian used-car market.

The analysis focuses on:
- Car price distribution
- Relationship between price and kilometers driven
- Impact of fuel type and transmission on pricing

Since Cars24 is a dynamic website, traditional static web scraping techniques are not sufficient.

In [3]:
page = requests.get(url)
page.status_code

200

-> requesting the website for content extraction and checking the webpage status.

In [4]:
page_html = BeautifulSoup(page.content, 'html.parser')

-> above we are extracting the content that present in the html code using "html.praser" inside BeautifulSoup

In [5]:
cars = page_html.find_all("div", class_= "styles_outer__NTVth")
len(cars)

20

-> in above one we are getting only 20 cars that is page defualt size car cards presented

In [6]:
from selenium import webdriver
import time

-> we are going to install selenium and webdriver for automation webpage checking and extracting.  
-> in this we have to scroll manually down more to get the more car cards for extraction.  
-> time for to check the web sleep if scrolling stops for given time the webpage will auto close.  
## Why Selenium Was Used

Cars24 is a JavaScript-driven dynamic website where car listings are loaded asynchronously using client-side rendering.

Key reasons Selenium was required:
- Page content is not present in the initial HTML response
- Data loads dynamically during scrolling
- BeautifulSoup and requests alone cannot access complete listing data

Selenium was used to:
- Automate browser behavior
- Handle dynamic content loading
- Scroll through listings and extract visible elements in real time


In [7]:
# from selenium.webdriver.common.by import By
# import time

# browser = webdriver.Chrome()
# browser.get(url)
# time.sleep()

# ini_time = time.time()
# MAX_TIME = 60   # seconds

# seen = set()
# all_cards = []

# while True:
#     cards = browser.find_elements(By.CLASS_NAME, "styles_outer__NTVth")

#     for card in cards:
#         html = card.get_attribute("outerHTML")
#         if html and html not in seen:
#             seen.add(html)
#             all_cards.append(html)

#     browser.execute_script("window.scrollBy(0, 1500);")
#     time.sleep(2)

#     if time.time() - ini_time > MAX_TIME:
#         break

# browser.quit()

# print("Total unique car cards (HTML):", len(all_cards))

In [8]:
from selenium.webdriver.common.by import By
import time

browser = webdriver.Chrome()
browser.get(url)
time.sleep(1)

seen = set()
all_cards = []

start_time = time.time()
MAX_TIME = 2500   # total runtime limit

while True:
    cards = browser.find_elements(By.CLASS_NAME, "styles_outer__NTVth")

    # 🔹 extraction logic from Code 1
    for card in cards:
        html = card.get_attribute("outerHTML")
        if html and html not in seen:
            seen.add(html)
            all_cards.append(html)

    # 🚀 fast scrolling from Code 2
    browser.execute_script("window.scrollBy(0, 1800);")
    time.sleep(0.6)

    # ⏹ stop condition (time-based, like Code 1)
    if time.time() - start_time > MAX_TIME:
        break

browser.quit()

print("Total unique cars captured", len(all_cards))


Total unique cars captured 3180


## Why Selenium Was Used

Cars24 is a JavaScript-driven dynamic website where content loads asynchronously. Selenium was required to handle dynamic rendering and scrolling behavior.

-> .Chrome() is used to open automation webpage in chrome  
-> sleep(3) is used for to check if the webpage is constant for 3 secs the webpage will autoclose  
-> ini_height is consisting of deafult loadpage size  
-> window.scroll used for autoscroll  
-> we are replacing the old(ini_height) with the new_height of the webpage  
-> browser.quit() is for autoclose of the webpage after 3 secs of sleep/ webpage constant 

## Challenges Faced – Virtualized DOM

Cars24 uses DOM virtualization, meaning only a limited number of listings are present in the HTML at any time. Traditional scroll-height logic does not work in this case.


In [9]:
len(all_cards)

3180

-> checking the number cards extracted from the webpage.

In [10]:
cars = []

for card_html in all_cards:
    soup = BeautifulSoup(card_html, "html.parser")
    cars.append(soup)
    
print("Total parsed car cards:", len(cars))

Total parsed car cards: 3180


-> checking the number of the cars extracted.

In [70]:
car = cars[0]
car

<div class="styles_outer__NTVth"><div class="sc-idXgbs gMoDyw"><div class="sc-kgTSHU kRKjic"><div class="sc-hHTYSq ikXFNv"><span class="sc-bcXHqh bAcffq" id="">2019 Maruti Celerio</span><span class="sc-bcXHqh bKVBht" id=""> <!-- -->VXI (O) CNG</span></div></div><div class="sc-fLcnxN cIFfdj"><ul class="sc-dIfARl hDolha"><div class="sc-hBxehJ gsPply"><p class="sc-bcXHqh kNDBvu" id="">85,350 km</p></div><div class="sc-hBxehJ gsPply"><p class="sc-bcXHqh kNDBvu" id="">CNG</p></div><div class="sc-hBxehJ gsPply"><p class="sc-bcXHqh kNDBvu" id="">Manual</p></div><div class="sc-hBxehJ gsPply"><p class="sc-bcXHqh kNDBvu" id="">HR-26</p></div></ul></div></div><div class="sc-bYMpWq isdbEF"><div class="styles_pricingDetail__Q_3hz"><div class="styles_flexItem__2z__J"><span class="sc-gswNZO lcwOwm"><p>EMI ₹6,432/m*</p></span></div><div class="styles_flexItem__2z__J"><div class="styles_priceWrap__VwWBV"><p class="sc-bcXHqh hnqWZb" id="">₹3.59L</p><p class="sc-bcXHqh hvRpEM" id="">₹3.29 lakh</p></div><

-> cars[0] for single car card details and html code deatils

In [12]:
print(car.prettify())          # gives you single car card details for extraction

<div class="styles_outer__NTVth">
 <div class="sc-idXgbs gMoDyw">
  <div class="sc-kgTSHU kRKjic">
   <div class="sc-hHTYSq ikXFNv">
    <span class="sc-bcXHqh bAcffq" id="">
     2019 Maruti Celerio
    </span>
    <span class="sc-bcXHqh bKVBht" id="">
     <!-- -->
     VXI (O) CNG
    </span>
   </div>
  </div>
  <div class="sc-fLcnxN cIFfdj">
   <ul class="sc-dIfARl hDolha">
    <div class="sc-hBxehJ gsPply">
     <p class="sc-bcXHqh kNDBvu" id="">
      85,350 km
     </p>
    </div>
    <div class="sc-hBxehJ gsPply">
     <p class="sc-bcXHqh kNDBvu" id="">
      CNG
     </p>
    </div>
    <div class="sc-hBxehJ gsPply">
     <p class="sc-bcXHqh kNDBvu" id="">
      Manual
     </p>
    </div>
    <div class="sc-hBxehJ gsPply">
     <p class="sc-bcXHqh kNDBvu" id="">
      HR-26
     </p>
    </div>
   </ul>
  </div>
 </div>
 <div class="sc-bYMpWq isdbEF">
  <div class="styles_pricingDetail__Q_3hz">
   <div class="styles_flexItem__2z__J">
    <span class="sc-gswNZO lcwOwm">
     

-> We will get the above html code format using .pretify()

In [52]:
name = car.find("span").get_text()
name

'2019 Maruti Celerio'

In [53]:
year_manufactured = name.split()[0]
brand = name.split()[1]
model = " ".join(name.split()[2:])

year_manufactured, brand, model

('2019', 'Maruti', 'Celerio')

In [54]:
lists = car.find_all('p')
lists

[<p class="sc-bcXHqh kNDBvu" id="">85,350 km</p>,
 <p class="sc-bcXHqh kNDBvu" id="">CNG</p>,
 <p class="sc-bcXHqh kNDBvu" id="">Manual</p>,
 <p class="sc-bcXHqh kNDBvu" id="">HR-26</p>,
 <p>EMI ₹6,432/m*</p>,
 <p class="sc-bcXHqh hnqWZb" id="">₹3.59L</p>,
 <p class="sc-bcXHqh hvRpEM" id="">₹3.29 lakh</p>,
 <p>+ other charges</p>]

In [55]:
len(lists)

8

In [56]:
km_tra = lists[0].get_text()
fuel = lists[1].get_text()
trans = lists[2].get_text()
rto = lists[3].get_text()
emi = lists[4].get_text().split()[1]
price = lists[5].get_text()
sale_price = lists[6].get_text()
km_tra, fuel, trans, rto, emi, price, sale_price

('85,350 km', 'CNG', 'Manual', 'HR-26', '₹6,432/m*', '₹3.59L', '₹3.29 lakh')

In [57]:
year_manufactured_lsist = []
brand_list = []
model_list = []
km_tra_list = []
fuel_list = []
trans_list = []
rto_list = []
emi_list = []
price_list = []
sale_price_list = []

In [8]:
rto_data = {
    "MH": "Maharashtra",
    "KA": "Karnataka",
    "GJ": "Gujarat",
    "HR": "Haryana",
    "TS": "Telangana",
    "TG": "Telangana",
    "DL": "Delhi",
    "WB": "West Bengal",
    "UP": "Uttar Pradesh",
    "BR": "Bihar",
    "MP": "Madhya Pradesh",
    "RJ": "Rajasthan",
    "CH": "Chandigarh",
    "TN": "Tamil Nadu",
    "KL": "Kerala",
    "AP": "Andhra Pradesh",
    "PB": "Punjab",
    "UK": "Uttarakhand",
    "HP": "Himachal Pradesh",
    "CG": "Chhattisgarh",
    "PY": "Puducherry",
    "DN": "Dadra and Nagar Haveli",
    "AR": "Arunachal Pradesh",
    "OD": "Odisha",
    "DD": "Daman and Diu",
    "AN": "Andaman and Nicobar Islands",
    "JH": "Jharkhand",
    "GA": "Goa",
    "MZ": "Mizoram",
    "NL": "Nagaland",
    "TR": "Tripura",
    "AS": "ASSAM",
    "SK": "Sikkim",
    "JK": "Jammu and Kashmir",
    "ML": "Meghalaya",
    "TE-MP": "Madhya Pradesh"
}


In [59]:
for i in cars:

    span = i.find("span")
    if not span:
        continue

    name = span.get_text()
    year_manufactured = name.split()[0]
    brand = name.split()[1]
    model = " ".join(name.split()[2:])

    lists = i.find_all('p')
    if len(lists) < 6:
        continue

    km_text = re.sub(r"[^\d]", "", lists[0].get_text())
    km_tra = int(km_text) if km_text else None

    fuel = lists[1].get_text()
    trans = lists[2].get_text()
    
    #  RTO mapping logic
    rto_raw = lists[3].get_text()          # e.g., "HR-26"
    rto_code = rto_raw.split("-")[0]       # "HR"
    rto = rto_data.get(rto_code, rto_raw)  # full state name or fallback

    emi_text = re.sub(r"[^\d]", "", lists[4].get_text())
    emi = int(emi_text) if emi_text else None

    # Base price
    price = (round(float(re.search(r"[\d.]+", lists[5].get_text()).group()) * 100000, 2)
        if re.search(r"[\d.]+", lists[5].get_text())
        else None)

    # Sale price -> fallback to price if NA
    sale_price = (round(float(re.search(r"[\d.]+", lists[6].get_text()).group()) * 100000, 2)
        if len(lists) > 6 and re.search(r"[\d.]+", lists[6].get_text())
        else price)

    year_manufactured_lsist.append(year_manufactured)
    brand_list.append(brand)
    model_list.append(model)
    km_tra_list.append(km_tra)
    fuel_list.append(fuel)
    trans_list.append(trans)
    rto_list.append(rto)
    emi_list.append(emi)
    sale_price_list.append(sale_price)

## Data Cleaning Using Regular Expressions

The scraped data contained currency symbols, units, and missing values. Regular expressions were used to extract and normalize numeric fields such as price, EMI, and kilometers driven.


In [60]:
model_list[:10]

['Celerio',
 'New Wagon-R',
 'NEW SANTRO',
 'NEXON',
 'SONET',
 'Celerio',
 'Elite i20',
 'Xcent',
 'Grand i10',
 'TUV300']

In [61]:
print("year_manufactured_lsist", len(year_manufactured_lsist))
print("brand_list", len(brand_list))
print("model_list", len(model_list))
print("km_tra_list", len(km_tra_list))
print("fuel_list", len(fuel_list))
print("trans_list", len(trans_list))
print("rto_list", len(rto_list))
print("emi_list", len(emi_list))
print("sale_price_list", len(sale_price_list))

year_manufactured_lsist 3180
brand_list 3180
model_list 3180
km_tra_list 3180
fuel_list 3180
trans_list 3180
rto_list 3180
emi_list 3180
sale_price_list 3180


In [62]:
df_cars24 = pd.DataFrame({'year_manufactured' : year_manufactured_lsist,
                       'brand' : brand_list,
                        'model' : model_list,
                        'km_tra' : km_tra_list,
                        'fuel' : fuel_list,
                        'trans' : trans_list,
                        'rto' : rto_list,
                        'emi' : emi_list,
                        'sale_price' : sale_price_list})

## Dataset Overview

The cleaned dataset was structured into a Pandas DataFrame for further analysis and visualization.


In [63]:
df_cars24.head()

,year_manufactured,brand,model,km_tra,fuel,trans,rto,emi,sale_price
0,2019,Maruti,Celerio,85350,CNG,Manual,Haryana,6432.0,329000.0
1,2019,Maruti,New Wagon-R,74374,Petrol,Manual,Haryana,5719.0,324000.0
2,2022,Hyundai,NEW SANTRO,91678,CNG,Manual,Haryana,6743.0,382000.0
3,2022,Tata,NEXON,21703,Petrol,Manual,Karnataka,12233.0,693000.0
4,2020,KIA,SONET,38236,Petrol,Manual,Uttar Pradesh,10203.0,578000.0


In [64]:
len(df_cars24)

3180

In [68]:
df_cars24.to_csv('Cars24.csv', index = False)

In [65]:
df_cars24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3180 entries, 0 to 3179
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year_manufactured  3180 non-null   object 
 1   brand              3180 non-null   object 
 2   model              3180 non-null   object 
 3   km_tra             3180 non-null   int64  
 4   fuel               3180 non-null   object 
 5   trans              3180 non-null   object 
 6   rto                3180 non-null   object 
 7   emi                3154 non-null   float64
 8   sale_price         3180 non-null   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 223.7+ KB


In [66]:
pd.isna(df_cars24).sum()

year_manufactured     0
brand                 0
model                 0
km_tra                0
fuel                  0
trans                 0
rto                   0
emi                  26
sale_price            0
dtype: int64

## Project Limitations

- Cars24 limits listings due to DOM virtualization
- Complete extraction from a single page is not possible
- Some listings contain missing information

Despite these limitations, the dataset provides meaningful insights into used-car pricing trends.


In [67]:
df_cars24.drop_duplicates()

,year_manufactured,brand,model,km_tra,fuel,trans,rto,emi,sale_price
0,2019,Maruti,Celerio,85350,CNG,Manual,Haryana,6432.0,329000.0
1,2019,Maruti,New Wagon-R,74374,Petrol,Manual,Haryana,5719.0,324000.0
2,2022,Hyundai,NEW SANTRO,91678,CNG,Manual,Haryana,6743.0,382000.0
3,2022,Tata,NEXON,21703,Petrol,Manual,Karnataka,12233.0,693000.0
4,2020,KIA,SONET,38236,Petrol,Manual,Uttar Pradesh,10203.0,578000.0
...,...,...,...,...,...,...,...,...,...
3175,2022,Nissan,MAGNITE,75684,Petrol,Manual,Karnataka,10463.0,593000.0
3176,2019,Hyundai,VENUE,63224,Petrol,Manual,Telangana,11383.0,645000.0
3177,2018,Honda,WR-V,76894,Diesel,Manual,Telangana,11671.0,597000.0
3178,2019,MG,HECTOR,83300,Petrol,Auto,Haryana,15376.0,898000.0


In [27]:
df = pd.read_csv('Cars24.csv')

In [28]:
df[df['rto'] == "TE-MP"]

,year_manufactured,brand,model,km_tra,fuel,trans,rto,emi,sale_price
1114,2022,Tata,NEXON,5827,Petrol,Auto,TE-MP,15838.0,925000.0
1746,2025,Tata,ALTROZ,5210,Petrol,Auto,TE-MP,14811.0,865000.0


While data analysis we have found a outlier of a state and replacing it here to avoild errors in data analysis

In [29]:
df['rto'] = df['rto'].replace({"TE-MP": "Madhya Pradesh"})

In [30]:
df[df['rto'] == "TE-MP"]

,year_manufactured,brand,model,km_tra,fuel,trans,rto,emi,sale_price


In [32]:
df.to_csv('Cars24.csv', index = False)